<a href="https://colab.research.google.com/github/towardsai/agentic-ai-engineering-course/blob/main/session_6_agents/structured_outputs_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Structured Outputs

## Overview

This notebook teaches you how to reliably extract structured data from Large Language Models (LLMs). Instead of getting free-form text responses, you'll learn to enforce specific data formats that your applications can easily parse and use.

**What You'll Learn:**
- Why structured outputs are crucial for reliable data extraction from LLMs
- How to enforce JSON output formats using prompt engineering techniques
- How to leverage Pydantic models for defining and validating complex data structures
- How to use Gemini's native structured output capabilities for maximum reliability

**What You'll Build:**
- A document metadata extractor that returns structured JSON
- A Pydantic-based validation system for LLM outputs
- A complete structured extraction pipeline using Gemini's native capabilities

We will use the `google-genai` library to interact with Google's Gemini models.

## 1. Setup

### Set Up Python Environment

Run the following command to install all the required packages to run this notebook.

In [ ]:
# Install dependencies
!pip install -q openai==2.15.0 google-genai==1.55.0 agentic-ai-engineering-course==0.4.6 google-auth==2.43.0 \
  opentelemetry-api==1.37.0 opentelemetry-sdk==1.37.0 opentelemetry-proto==1.37.0 \
  opentelemetry-exporter-otlp-proto-http==1.37.0 opentelemetry-exporter-otlp-proto-common==1.37.0 \
  jedi==0.19.2

%pip check

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.

### Configure Gemini API


In [ ]:
import os
import getpass

# Try to get API key from Colab secrets, fall back to manual input
try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except Exception:
    if not os.getenv("GOOGLE_API_KEY"):
        os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter Google API Key: ")

print("[OK] Packages installed and API key configured!")

### Import Key Packages

In [ ]:
import json

from google import genai
from google.genai import types
from pydantic import BaseModel, Field

from utils import pretty_print

### Initialize the Gemini Client

In [ ]:
client = genai.Client()

### Define Constants

We will use the `gemini-2.5-flash` model, which is fast and cost-effective:

In [ ]:
MODEL_ID = "gemini-2.5-flash"

## 2. Implementing structured outputs from scratch using JSON

Sometimes, you don't need the LLM to take an action, but you need its output in a specific, machine-readable format. Forcing the output to be JSON is a common way to achieve this.

We can instruct the model to do this by **prompting** clearly describing the desired JSON structure in the prompt.

### Example: Extracting Metadata from a Document

Let's imagine we have a markdown document and we want to extract key information like a summary, tags, and keywords into a clean JSON object.

In [ ]:
DOCUMENT = """
# Q3 2023 Financial Performance Analysis

The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement,
beating market expectations. These impressive results reflect our successful product strategy
and strong market positioning.

Our core business segments demonstrated remarkable resilience, with digital services leading
the growth at 25% year-over-year. The expansion into new markets has proven particularly
successful, contributing to 30% of the total revenue increase.

Customer acquisition costs decreased by 10% while retention rates improved to 92%,
marking our best performance to date. These metrics, combined with our healthy cash flow
position, provide a strong foundation for continued growth into Q4 and beyond.
"""

prompt = f"""
Analyze the following document and extract metadata from it.
The output must be a single, valid JSON object with the following structure:
<json>
{{
    "summary": "A concise summary of the article.",
    "tags": ["list", "of", "relevant", "tags"],
    "keywords": ["list", "of", "key", "concepts"],
    "quarter": "Q...",
    "growth_rate": "...%",
}}
</json>

Here is the document:
<document>
{DOCUMENT}
</document>
"""

response = client.models.generate_content(model=MODEL_ID, contents=prompt)

pretty_print.wrapped(text=response.text, title="Raw LLM Output", indent=2)

In [ ]:
def extract_json_from_response(response: str) -> dict:
    """
    Extracts JSON from a response string that is wrapped in <json> or ```json tags.
    """

    response = response.replace("<json>", "").replace("</json>", "")
    response = response.replace("```json", "").replace("```", "")

    return json.loads(response)

You can now reliably parse the JSON string:

In [ ]:
parsed_response = extract_json_from_response(response.text)
pretty_print.wrapped(
    text=[f"Type of the parsed response: `{type(parsed_response)}`", json.dumps(parsed_response, indent=2)],
    title="Parsed JSON Object",
    indent=2,
)

## 3. Implementing structured outputs from scratch using Pydantic

While prompting for JSON is effective, it can be fragile. A more robust and modern approach is to use **Pydantic**. Pydantic allows you to define data structures as Python classes. This gives you:

- **A single source of truth**: The Pydantic model defines the structure.
- **Automatic schema generation**: You can easily generate a JSON Schema from the model.
- **Data validation**: You can validate the LLM's output against the model to ensure it conforms to the expected structure and types.

Let's recreate the previous example using Pydantic.

In [ ]:
class DocumentMetadata(BaseModel):
    """A class to hold structured metadata for a document."""

    summary: str = Field(description="A concise, 1-2 sentence summary of the document.")
    tags: list[str] = Field(description="A list of 3-5 high-level tags relevant to the document.")
    keywords: list[str] = Field(description="A list of specific keywords or concepts mentioned.")
    quarter: str = Field(description="The quarter of the financial year described in the document (e.g, Q3 2023).")
    growth_rate: str = Field(description="The growth rate of the company described in the document (e.g, 10%).")

### Injecting Pydantic Schema into the Prompt

We can generate a JSON Schema from our Pydantic model and inject it directly into the prompt. This is a more formal way of telling the LLM what structure to follow.

Note how, along with the field type, we can leverage the Field description automatically to clearly specify to the LLM what each field means.

In [ ]:
schema = DocumentMetadata.model_json_schema()
schema

In [ ]:
prompt = f"""
Please analyze the following document and extract metadata from it.
The output must be a single, valid JSON object that conforms to the following JSON Schema:
<json>
{json.dumps(schema, indent=2)}
</json>

Here is the document:
<document>
{DOCUMENT}
</document>
"""

response = client.models.generate_content(model=MODEL_ID, contents=prompt)

parsed_response = extract_json_from_response(response.text)

pretty_print.wrapped(
    text=[f"Type of the parsed response: `{type(parsed_response)}`", json.dumps(parsed_response, indent=2)],
    title="Parsed JSON Object",
    indent=2,
)

As you can see, conceptually, the results are the same. But now, we can easily validate the output with Pydantic:

In [ ]:
try:
    document_metadata = DocumentMetadata.model_validate(parsed_response)
    print("\nValidation successful!")

    pretty_print.wrapped(
        ["Type of the validated response: `{type(document_metadata)}`", document_metadata.model_dump_json(indent=2)],
        title="Pydantic Validated Object",
        indent=2,
    )
except Exception as e:
    print(f"\nValidation failed: {e}")

### Exercise: Implement Structured Extraction with Native Gemini Support

Now it's your turn! In this exercise, you'll use Gemini's native structured output capabilities to extract metadata from the document.

**Learning Goal:**
Learn how to leverage Gemini's built-in support for structured outputs using `GenerateContentConfig` and Pydantic schemas.

**What You Need to Implement:**
1. Create a configuration object that enforces JSON output with schema validation
2. Write a simple prompt that wraps the document
3. Call the model with the configuration
4. Display the validated Pydantic object

**Key Concepts:**
- `types.GenerateContentConfig` - Configuration class for model generation
- `response_mime_type="application/json"` - Forces JSON output format
- `response_schema=DocumentMetadata` - Specifies the Pydantic model to validate against
- `response.parsed` - Returns the already-validated Pydantic object (no manual parsing needed!)

In [ ]:
# EXERCISE: Structured extraction with a schema

# Hint: start from these variables that already exist above:
#   - client, MODEL_ID, DOCUMENT, DocumentMetadata, types, pretty_print

# TODO: your code here
raise NotImplementedError('Fill in the config, prompt, and model call.')


### Validation Check

Run the cell below to verify your implementation works correctly:

In [ ]:
# # Validation: Check if your implementation is correct

# try:
#     # Check if response exists and has parsed attribute
#     assert 'response' in dir(), "❌ 'response' variable not found. Make sure you called generate_content()."
#     assert hasattr(response, 'parsed'), "❌ Response doesn't have 'parsed' attribute. Check your config."

#     # Check if parsed is a DocumentMetadata instance
#     assert isinstance(response.parsed, DocumentMetadata), f"❌ Expected DocumentMetadata, got {type(response.parsed)}"

#     # Check if all required fields are present
#     required_fields = ['summary', 'tags', 'keywords', 'quarter', 'growth_rate']
#     for field in required_fields:
#         assert hasattr(response.parsed, field), f"❌ Missing field: {field}"
#         value = getattr(response.parsed, field)
#         assert value is not None and value != "", f"❌ Field '{field}' is empty"

#     # Check data types
#     assert isinstance(response.parsed.summary, str), "❌ 'summary' should be a string"
#     assert isinstance(response.parsed.tags, list), "❌ 'tags' should be a list"
#     assert isinstance(response.parsed.keywords, list), "❌ 'keywords' should be a list"
#     assert len(response.parsed.tags) >= 3, "❌ Should have at least 3 tags"

#     print("✅ All validation checks passed!")
#     print("\n🎉 Congratulations! Your implementation is correct.")
#     print("\nYou've successfully:")
#     print("  - Created a proper GenerateContentConfig")
#     print("  - Used response_mime_type and response_schema correctly")
#     print("  - Extracted structured data with native Gemini support")
#     print("  - Obtained a validated Pydantic object directly")

# except AssertionError as e:
#     print(f"\n{e}")
#     print("\n💡 Tips:")
#     print("  - Make sure you create the config with both response_mime_type and response_schema")
#     print("  - Pass the config to generate_content() using config=your_config")
#     print("  - Access the parsed result using response.parsed")
# except Exception as e:
#     print(f"\n❌ Error: {e}")
#     print("\n💡 Make sure you've completed the exercise cell above first.")

## Exercises

1. Define a new Pydantic schema to extract **people mentioned** in the document (name, role, sentiment), and re-run the structured extraction.
2. Add a field for `risk_factors: list[str]` and compare extraction quality with and without examples in the prompt.
3. Wrap the extraction call in a retry loop that re-prompts the model if JSON validation fails.
